# Git Learning examples walkthrough
Some examples of using Git for version control!  Contents:

[New local repository](#New-Local-repository)
- `init` new local repo
- `add` files
- `commit` changes
- see commit `log`

[Have a remote repository](#Have-a-remote-repository)
- walk through GitHub repo creation
- local repo `remote add`
- git `push`
- see commit in GitHub
- modification/new doc `add`, `commit`, `push`, and see subsequent commits in GitHub

[Illustrate distinctness of local git repo and GitHub remote repo](#Illustrate-distinctness-of-local-git-repo-and-GitHub-remote-repo)
- delete local clone of repo
- remote repo persists in GitHub!

[Collaboration](#Collaboration)
- `clone` of repo to another filesystem location or computer
- show history in GitHub (diffs, etc)

[Some Git technical details](#Some-Git-technical-details)
- Git verbs and what they do on filesystem -- a `.git` graphic

---
## New Local repository
Creating a new, local, version controlled repository. In this example, we'll have a folder of existing scripts (made on the fly) that will be our sample contents of interest.

In [241]:
## setup for demo some test folder / files / etc, and set location as the new folder
($oSampleProjectDirectory = New-Item -ItemType Directory -Path C:\Temp\GitTesting\sampleProject0-$(New-Guid)) | Set-Location
## create a few, clearly useful scripts that represent some body of code/scripts/documents that are the items we want in our repo
Get-Command -Module Microsoft.PowerShell.Utility, Microsoft.PowerShell.Management -Verb Get | Get-Random -Count 6 |
    ForEach-Object {
        Set-Content -Path (Join-Path -Path $oSampleProjectDirectory.FullName -ChildPath ("{0}-My{1}.ps1" -f $_.Verb, $_.Noun)) `
            -Value "# some cool script we use all the time", $_.Name -Verbose
    }

## see the contents of our sample project folder
$oSampleProjectDirectory | Get-ChildItem -OutVariable arrOurScriptFilesInfo

VERBOSE: Performing the operation "Set Content" on target "Path: C:\Temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1\Get-MyChildItem.ps1".
VERBOSE: Performing the operation "Set Content" on target "Path: C:\Temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1\Get-MyUnique.ps1".
VERBOSE: Performing the operation "Set Content" on target "Path: C:\Temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1\Get-MyPSDrive.ps1".
VERBOSE: Performing the operation "Set Content" on target "Path: C:\Temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1\Get-MyVariable.ps1".
VERBOSE: Performing the operation "Set Content" on target "Path: C:\Temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1\Get-MyLocation.ps1".
VERBOSE: Performing the operation "Set Content" on target "Path: C:\Temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1\Get-MyRunspaceDebug.ps1".


    Directory: C:\temp\GitTesting\sampleProject0-744c

### Initialize new local repo
Here we have a now a local folder of our super useful scripts and possibly accompanying docs whose versions we want to start controlling via Git. In order to make a new Git version control repository, we initialize said new repository with the verb [init](https://git-scm.com/docs/git-init). There is a parameter supplied here, too, to specify the name of the primary "branch" of the repository (we'll call it `main`)

In [242]:
(Get-Location).Path ## show that we are currently in our folder of files that we want to use for our new repo
git init --initial-branch main

Microsoft.PowerShell.Core\FileSystem::C:\temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1
Initialized empty Git repository in C:/temp/GitTesting/sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1/.git/


### Status
To see the status of our local repository, we use the `status` verb. This shows us things like new ("untracked") / modified / deleted files in the working directory, the branch on which we are, along with contextual info about what next

In [243]:
## get the git status
git status

On branch main

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Get-MyChildItem.ps1
	Get-MyLocation.ps1
	Get-MyPSDrive.ps1
	Get-MyRunspaceDebug.ps1
	Get-MyUnique.ps1
	Get-MyVariable.ps1

nothing added to commit but untracked files present (use "git add" to track)


Here we see that there are no commits, yet, that we have a bunch of new ("untracked") files, and that we can do something like `git add` to stage some files

### Add
In order to include new and modified files in our repository, we first `add` them to the list of items to be committed

In [244]:
## add some files to be a part of our repository
git add $arrOurScriptFilesInfo --verbose

add 'Get-MyChildItem.ps1'
add 'Get-MyLocation.ps1'
add 'Get-MyPSDrive.ps1'
add 'Get-MyRunspaceDebug.ps1'
add 'Get-MyUnique.ps1'
add 'Get-MyVariable.ps1'


Here, the variable `$arrOurScriptFilesInfo` holds the `FileInfo` info for all of the files in our working directory. We can explicitly name the files to add, use wildcards, take input from other commands, etc.

Let's check the repository status again, after having added those files

In [245]:
git status

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   Get-MyChildItem.ps1
	new file:   Get-MyLocation.ps1
	new file:   Get-MyPSDrive.ps1
	new file:   Get-MyRunspaceDebug.ps1
	new file:   Get-MyUnique.ps1
	new file:   Get-MyVariable.ps1



Progress -- we have some changes to be committed, now! Look at all those new files

### Commit
Once we have added ("staged") some changes for our repository, we then `commit` those changes. We want to have a meaningful message about what we achieved in this commit, so that others can quickly/easily understand the purpose of the commit (should not describe every character that was altered -- there are `diff`s for that)

In [246]:
git commit --message "Initial commit to get our valuable scripts into our new repo"

[main (root-commit) 278e250] Initial commit to get our valuable scripts into our new repo
 6 files changed, 12 insertions(+)
 create mode 100644 Get-MyChildItem.ps1
 create mode 100644 Get-MyLocation.ps1
 create mode 100644 Get-MyPSDrive.ps1
 create mode 100644 Get-MyRunspaceDebug.ps1
 create mode 100644 Get-MyUnique.ps1
 create mode 100644 Get-MyVariable.ps1


### Log
We are building a history for all of the things in our repository. To see info about when we committed such changes, we use the git `log`. Let's see what our repository's log looks like now

In [247]:
git log

commit 278e25020d2834084c2a35e85efcaa6e0eaf29ca
Author: MTBoren <technology@ddayinc.com>
Date:   Tue Jun 15 20:35:49 2021 -0400

    Initial commit to get our valuable scripts into our new repo


Alright -- we have a legit commit with things like the author info, the date/time of the commit, the commit ID, and the commit message -- yess!

---
## Have a remote repository
Using a local repository for controlling version of our important things is great. For many reasons, though, we want to then leverage the power of adding a _remote_ repository at which to house an authoritative copy of our repository/code/documents -- the source of truth!

For this, we will create a new remote repository on the wildly popular GitHub social coding platform

### GitHub remote repository creation
Here we will create a new remote repository on GitHub. There are plenty of places on GitHub that show how to do this. Particularly, you might have a look at [this walkthrough](https://docs.github.com/en/github/importing-your-projects-to-github/importing-source-code-to-github/adding-an-existing-project-to-github-using-the-command-line) for if/when you feel like going a bit deeper on this via the GitHub docs.

### Configure local repository to have a remote repository
Now that we have a remote repository, we set up our local repository to use said remote as the destination to which we `push` committed changes. This is just a single configuration update to our local repository. And, thanks GitHub -- they are sure to provide Quick Setup tips after the completion of the remote repository creation -- handy! We'll use part of that next

Here we add a remote repository named `origin` to our local repository. `origin` is just a name for the remote repository -- and alias if you will. While it could be any name, the standard is to use `origin`

In [248]:
git remote add origin https://github.com/mtboren/SuperGoodProject.git
git remote --verbose

origin	https://github.com/mtboren/SuperGoodProject.git (fetch)
origin	https://github.com/mtboren/SuperGoodProject.git (push)


Great -- our local repository is configured to use the new GitHub repository as its remote named `origin`. Now we `push` it

### Push
We have a local repo with a commit, and we want to put that commit in a place that it can live on forever, regardless of the fate of the machine on which we initially created the code/docs. Also, we want to make available to all of our peers/collaborators the hot new updates that we made in our local repository.

We use `push` to, well, push our local commits up to our configured remote repo. And, as a one-time thing per branch/remote combo, we have to set the "upstream" location to which we want to `push` our commits. That's why the `--set-upstream` parameter in this initial `push` call (subsequent `push` calls will not need this parameter)

In [249]:
git push --set-upstream origin main

Branch 'main' set up to track remote branch 'main' from 'origin'.
To https://github.com/mtboren/SuperGoodProject.git
 * [new branch]      main -> main


Success! Let's check that local repo `status`, again

In [250]:
git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Great! Everything locally is up to date with the remote, and our commit is now also stored in our source of truth -- GitHub

### See commit in GitHub
Alright, everyone said, "it just works", but let's have a look-see back at GitHub to see what our new remote repository looks like after the `push`.  Ah, not bad, not bad!

### Repeat: File add/modify, commit, push
Versions time -- we want to add some things to our repo, like:
- a proper ReadMe that describes the purpose of the repo
- some new feature to an existing script

In [251]:
## 'enhance' some script with the feature for which someone, for some important reason, asked
$oSampleProjectDirectory | Get-ChildItem -Filter *.ps1 | Get-Random -OutVariable oFileInfoOfScriptToEnhance |
    Add-Content -Value 'Write-Verbose "Script ran from $PSScriptRoot, and finished at $(Get-Date)"' -Verbose

VERBOSE: Performing the operation "Add Content" on target "Path: Microsoft.PowerShell.Core\FileSystem::C:\temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1\Get-MyPSDrive.ps1".


K, let's see what is the difference (`diff`) between the now modified script and the original version of it in the local repository

In [252]:
git diff

diff --git a/Get-MyPSDrive.ps1 b/Get-MyPSDrive.ps1
index 0ea2e1d..9e8050e 100644
--- a/Get-MyPSDrive.ps1
+++ b/Get-MyPSDrive.ps1
@@ -1,2 +1,3 @@
 # some cool script we use all the time
 Get-PSDrive
+Write-Verbose "Script ran from $PSScriptRoot, and finished at $(Get-Date)"


We'll also add a ReadMe with descriptive info about this repo, so people know what the world it does

In [253]:
Set-Content -Path ReadMe.md -Value "# The Valuable and Important code for Wonder Team X", "These are some scripts that we use for managing our environment" -Verbose

VERBOSE: Performing the operation "Set Content" on target "Path: Microsoft.PowerShell.Core\FileSystem::C:\temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1\ReadMe.md".


And then, we'll `add` our modifications to the list of things to be committed, `commit` the modifications, and then `push` the commit to GitHub

In [254]:
git add $oFileInfoOfScriptToEnhance ReadMe.md --verbose

add 'Get-MyPSDrive.ps1'
add 'ReadMe.md'


Check once more our status, make sure that the things that we want to commit are the only things staged for commit

In [255]:
git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   Get-MyPSDrive.ps1
	new file:   ReadMe.md



Beauty! Now, to commit and push

In [256]:
## commit with a meaningful commit message, that shows that it fixes Issue #1 (assuming that someone had submitted an Issue requesting the feature)
git commit --message "Add feature to our script (fix #1), add ReadMe"

[main 296bc54] Add feature to our script (fix #1), add ReadMe
 2 files changed, 3 insertions(+)
 create mode 100644 ReadMe.md


In [257]:
## push to configured remote repo "origin" by default
git push

To https://github.com/mtboren/SuperGoodProject.git
   278e250..296bc54  main -> main


Now, to see those updates in GitHub. Notice the auto-rendered ReadMe -- handy!

---
## Illustrate distinctness of local git repo and GitHub remote repo
Local- and remote repositories have a relationship via the local repo's configured `remote`, but they are two, distinct things. Deleting one does nothing to the other, for example.
### Delete local repo
In fact, let's do that: we'll delete the local repository and working directory, and then see what is the state of the remote repository

In [258]:
## set location somewhere external to the local repo's working directory, so we can delete said directory
Set-Location -Path $env:TEMP -PassThru


Path
----
C:\Users\mtboren\AppData\Local\Temp



In [259]:
## then, let's delete the local repo and working directory (have to -Force due to the state of some of the files in the repository directory)
Remove-Item -Path $oSampleProjectDirectory -Force


Confirm
The item at C:\Temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1 has children and the Recurse parameter was not specified. If you continue, all children will be removed with the item. Are you sure you want to continue?
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  


In [260]:
## and, show that said folder and repo are actually gone
New-Object -Type PSObject -Property ([ordered]@{Path = $oSampleProjectDirectory; bExists = Test-Path -Path $oSampleProjectDirectory})


Path                                                                   bExists
----                                                                   -------
C:\Temp\GitTesting\sampleProject0-744c2a05-9356-4c9e-bc8c-09735b9ec8e1   False



Gone -- local repo is deleted from disk. But is our remote repo (our source of truth) still intact? We'll see..

### Remote repo persists in GitHub
We've now deleted the local repo and working directory from the local filesystem. We're _pretty_ confident that the remote is still intact, but let's confirm (goto https://github.com/mtboren/SuperGoodProject.git now!)

All good? Whew 😍. So, this means that:
- if we've vetted the remote service (GitHub), we can depend on the service as our persistent source of truth
- we can then free ourselves of having to treat local copies of code as wonderful snowflakes to have and to hold, to make copies as backup, to care about the things on the local filesystem beyond the last `commit`/`push` -- liberated! 🗽
- we can delete local copies at will (after we've `push`ed) -- the remote is the source of truth!
- we should always be able to recreate a local copy of the repo at a `clone`'s notice -- we'll see about cloning in the Collaboration section below

---
## Collaboration
Now that we have our code in some remote repository, we are all set to collaborate with all the brilliant minds that have even read access to said remote repository -- discuss issues, merge their code updates (either directly, or with review gates in place), plan projects, publish releases, maintain a wiki, etc!
### Work on the latest: Clone of a remote repo
When we're ready to start crunching code and we don't already have a local copy of a repo, we can `clone` the repo down from the remote

In [261]:
## clone from remote to a subfolder in the specified path
$strDirectoryOfNewRepoClone = "${env:Temp}\superGoodProj_clone-$(New-Guid)"
git clone https://github.com/mtboren/SuperGoodProject.git $strDirectoryOfNewRepoClone
Set-Location -Path $strDirectoryOfNewRepoClone

Cloning into 'C:\Users\mtboren\AppData\Local\Temp\superGoodProj_clone-3e6138f2-761b-4211-88f7-ae2b4fd55ff2'...


In [262]:
Get-ChildItem -Path $strDirectoryOfNewRepoClone -Force



    Directory: 
C:\Users\mtboren\AppData\Local\Temp\superGoodProj_clone-3e6138f2-761b-4211-88f7-ae2b4fd55ff2

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d--h-           6/15/2021  8:39 PM                .git
-a---           6/15/2021  8:39 PM             55 Get-MyChildItem.ps1
-a---           6/15/2021  8:39 PM             54 Get-MyLocation.ps1
-a---           6/15/2021  8:39 PM            129 Get-MyPSDrive.ps1
-a---           6/15/2021  8:39 PM             59 Get-MyRunspaceDebug.ps1
-a---           6/15/2021  8:39 PM             52 Get-MyUnique.ps1
-a---           6/15/2021  8:39 PM             54 Get-MyVariable.ps1
-a---           6/15/2021  8:39 PM            118 ReadMe.md



Bam! We now have a local clone of the remote, and we're ready to start making the project ever better! We proceed to do our modifications in the working directory, then the ol' `add` / `commit` / `push` routine that we know and love :heart: by now!

In [263]:
## edit another file (add help to it)
$strDirectoryOfNewRepoClone | Get-ChildItem -Filter *.ps1 | Get-Random -OutVariable oFileInfoOfAnotherScriptToEnhance |
    Add-Content -Value "<# .Description", "Get the important things", ".Example", "$($oFileInfoOfAnotherScriptToEnhance.Name) -Verbose",
    "$($oFileInfoOfAnotherScriptToEnhance.BaseName) for all the things, for when you really need to do so", "#>" -Verbose

## and, delete one file that our team decided we don't need anymore
$strDirectoryOfNewRepoClone | Get-ChildItem -Filter *.ps1 | Get-Random -OutVariable oFileInfoOfScriptToRemove | Remove-Item -Verbose

VERBOSE: Performing the operation "Add Content" on target "Path: Microsoft.PowerShell.Core\FileSystem::C:\Users\mtboren\AppData\Local\Temp\superGoodProj_clone-3e6138f2-761b-4211-88f7-ae2b4fd55ff2\Get-MyLocation.ps1".
VERBOSE: Performing the operation "Remove File" on target "C:\Users\mtboren\AppData\Local\Temp\superGoodProj_clone-3e6138f2-761b-4211-88f7-ae2b4fd55ff2\Get-MyChildItem.ps1".


In [264]:
## we added some initial Help to one of the scripts (we should have it in _all_ of our scripts!)
Get-Help -Full $oFileInfoOfAnotherScriptToEnhance[0]


NAME
    C:\Users\mtboren\AppData\Local\Temp\superGoodProj_clone-3e6138f2-761b-4211-88f7-ae2b4fd55ff2\Ge
    t-MyLocation.ps1
    
SYNOPSIS
    
    
SYNTAX
    C:\Users\mtboren\AppData\Local\Temp\superGoodProj_clone-3e6138f2-761b-4211-88f7-ae2b4fd55ff2\Ge
    t-MyLocation.ps1 [<CommonParameters>]
    
    
DESCRIPTION
    Get the important things
    

PARAMETERS
    <CommonParameters>
        This cmdlet supports the common parameters: Verbose, Debug,
        ErrorAction, ErrorVariable, WarningAction, WarningVariable,
        OutBuffer, PipelineVariable, and OutVariable. For more information, see
        about_CommonParameters (https://go.microsoft.com/fwlink/?LinkID=113216). 
    
INPUTS
    
OUTPUTS
    
    -------------------------- EXAMPLE 1 --------------------------
    
    PS > Get-MyItemProperty.ps1 -Verbose
    Get-MyItemProperty for all the things, for when you really need to do so
    
    
    
    
    
    
    
RELATED LINKS




In [265]:
## stage changes
git add $oFileInfoOfAnotherScriptToEnhance --verbose
git rm $oFileInfoOfScriptToRemove

add 'Get-MyLocation.ps1'
rm 'Get-MyChildItem.ps1'


In [266]:
## before commit, check the status and see the diff
git status
git diff --staged

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	deleted:    Get-MyChildItem.ps1
	modified:   Get-MyLocation.ps1

diff --git a/Get-MyChildItem.ps1 b/Get-MyChildItem.ps1
deleted file mode 100644
index c531526..0000000
--- a/Get-MyChildItem.ps1
+++ /dev/null
@@ -1,2 +0,0 @@
-# some cool script we use all the time
-Get-ChildItem
diff --git a/Get-MyLocation.ps1 b/Get-MyLocation.ps1
index 4db7846..91acf91 100644
--- a/Get-MyLocation.ps1
+++ b/Get-MyLocation.ps1
@@ -1,2 +1,8 @@
 # some cool script we use all the time
 Get-Location
+<# .Description
+Get the important things
+.Example
+Get-MyItemProperty.ps1 -Verbose
+Get-MyItemProperty for all the things, for when you really need to do so
+#>


In [267]:
## commit, push
git commit -m "Added help, removed obsolete script (fix #2)"
git push

[main 1fa84a7] Added help, removed obsolete script (fix #2)
 2 files changed, 6 insertions(+), 2 deletions(-)
 delete mode 100644 Get-MyChildItem.ps1
To https://github.com/mtboren/SuperGoodProject.git
   296bc54..1fa84a7  main -> main


¡Voila! We cloned down a local copy of the repo, made and reviewed our changes, added/committed, and the pushed the commit up to the origin / source of truth! Collaboration in full effect.

When done, we'll clean up the temporary local clone of the repo, since all of our wonderful adds (and deletes) have been pushed to the trusty remote reposity (GitHub). Yess!

In [268]:
## clean up the temporary local clone, now that we've pushed our changes
Set-Location $env:TEMP
Remove-Item -Path $strDirectoryOfNewRepoClone -Force


Confirm
The item at C:\Users\mtboren\AppData\Local\Temp\superGoodProj_clone-3e6138f2-761b-4211-88f7-ae2b4fd55ff2 has children and the Recurse parameter was not specified. If you continue, all children will be removed with the item. Are you sure you want to continue?
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  y


### History in GitHub (diffs, etc)
Coding and versioning and collaborating -- oh, my!  Let's have a look at the repo, its commits, the history, etc: https://github.com/mtboren/SuperGoodProject.git

---
## Some Git technical details
Some details about the elements that comprise the version controlled repo, the working files on disk, etc.
### Git verbs and what they do on filesystem: a .git graphic
Have a look at the basic git operations graphic: ![git operations graphic](./resources/git-operations.png)